take a look at this guides:
- (Korzynski, Mazurek, Krzypkowska, & Kurasinski, 2023)

Use Template pattern for output format promtp
- (White, Fu, Hays, Sandborn, Olea, Gilbert, Elnashar, Spencer-Smith, & Schmidt, 2023)

Use PB Smells form Qamar et Al 2021 (see overleaf chapter)

In [1]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

# Initial Prompt

In [2]:
init_temp = """### Instruction ###
Act as a professional {role} working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Your outputs should correspond to those a {role} would regarding the ticket.
We want to revise a ticket for compliance with these best practices and offer recommendations if needed.

### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp, and content.
Components: Project components to which the issue belongs.
CreatedDate: The time and date the issue was created.
Creator: The person who created the issue.
Description: A detailed description of the issue.
IssueLinks: A list of links to related issues.
IssueType: The issue's purpose within the organization.
Labels: Labels to which this issue relates.
Priority: The issue importance in relation to other issues.
Project: The parent project to which the issue belongs.
Reporter: The person who found/reported the issue.
Resolution: A record of the issue's resolution, once resolved or closed.
ResolvedDate: The time and date the issue was resolved.
Status: The stage the issue is currently at in its lifecycle.
Summary: A brief one-line summary of the issue.
TimeSpent: Amount of time spent working on the issue.
VersionsAffected: The versions of the project affected by the issue.
VersionsFixed: Project versions in which the issue was (or will be) fixed.

### Best Practice ###
The best practice to check is {best_practice}"""
init_prompt = PromptTemplate(template= init_temp, input_variables= ["role", "best_practice"])

init_prompt.save("prompts/init/initPrompt_V2.1.0.json")

# Update Prompt (Best Practice)

#### Prompt

In [ ]:
update_temp ="""Best Practice: Update Missing/Outdated Fields
During the evolution of a ticket, some fields may have become outdated or missing.
You could find the needed information in the "Summary", "Description", or "Comments" fields.
Get contextual knowledge about the ticket through the "Summary", "Description", and "Comments" fields.
If someone specifically mentions or suggests a change for the ticket in the "Comments" field, update the ticket accordingly.
The change can be provided by a human or as a message from a programm.
Ignore error messages, logs and quotes within the comment.
Possible keywords for such a change are: "edit", "change", "move", "mark", "fix", "set", "update", "modify", "correct", "adjust", "resolve" and "should".
Pay particular attention to the most recent comments.

Return only mentioned or suggested changes from the "Comments" field."""

update_prompt = PromptTemplate(template= update_temp, input_variables= [])
update_prompt.save("prompts/update/updatePrompt_V1.26.0.json")

#### Output format prompt

In [ ]:
update_format_temp ="""### Instruction ###
You receive changes and recommendations made to fields of a ticket from an issue tracker.
Return the data  in the following format as JSON-format:
"<violation_predicted>": "<Set to TRUE, if a change was detected and made, otherwise FALSE>",
"fields": "<Contains the list for the receives changes, icnluding <field>, <recommendation>, and <reason>>"
"field": "<name of updated field>", 
"changed_to": "<the new input of the field after update, without any explanation>", 
"reason": "<reason for the update>"

### Context ###
Data: {revised_ticket}"""

update_format_prompt = PromptTemplate(template= update_format_temp, input_variables= ["revised_ticket"])

update_format_prompt.save("prompts/update/updateFormatPrompt_V1.12.0.json")

# Summary Length Prompt (Best Practice)


examples:
- https://bugzilla.mozilla.org/page.cgi?id=bug-writing.html
- https://bugs.eclipse.org/bugs/page.cgi?id=bug-writing.html

#### Zero-Shot Prompt

In [5]:
sumLen_temp = """'Summary Length':
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}"""

sumLen_prompt = PromptTemplate(template= sumLen_temp, input_variables= ["min", "max", "ticket"])

sumLen_prompt.save("prompts/summary/summaryLengthPrompt_V3.2.0.json")

#### Zero-Shot CoT Prompt

In [ ]:
sumLen_0ShotCoT_temp = """Best Practice: Summary Length
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.
Let's think step by step."""

sumLen_0ShotCoT_prompt = PromptTemplate(template= sumLen_0ShotCoT_temp, input_variables= ["min", "max"])

sumLen_0ShotCoT_prompt.save("prompts/summary/summaryLengthPrompt_0ShotCoT_V1.0.0.json")

#### Few-Shot Prompt

In [3]:
examples = [
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Sourcetree crashes',
'Description':'Source tree crashes everytime i open , I have uninstalled and installed many times',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'Git'}}""",
        "output": """The current summary is "Sourcetree crashes", which consists of 18 characters. This is below the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Sourcetree crashes upon opening repeatedly"

This revised summary is 42 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Opening and closing Sourcetree causes Pageant to crash\/close',
'Description':'I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready.\r\n\r\nDidn't quite notice it at first, so not sure which version it started at, but Sourcetree now causes Pageant to crash\/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason.\r\n\r\nWhenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. Sourcetree shouldn't close or cause any other applications to crash when it opens or when it closes.\r\n\r\nIt happens on all 3 computers of mine. All running latest Windows, latest Pageant, and latest Sourcetree.',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'General'}}""",
        "output": """The current summary is "Opening and closing Sourcetree causes Pageant to crash\/close" which consists of 61 characters. This is within the best practice range of 39 to 70 characters.

No summary has to be recommended. 

The original summary is compliant with the best practice for summary length and does not require any changes."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Cannot change title of News tool',
'Description':'After initially setting up the News tool during worksite setup, I tried to go to the option page (NEWS-2) and change the title of the tool. After I made the change, the title of the tool did not update, even after I logged out and logged back in. However, when I opened up options, the new title that I had entered was still specified.\n\nI tried to change the URL of the feed as well as the title at the same time, and even though the new feed was displayed properly, the title of the tool remained what I had initially set it to during worksite setup.',
'VersionsAffected':'2.0',
'IssueType':'Bug',
'Project':'Sakai',
'Components':'News (RSS)'}}""",
        "output": """The current summary is "Cannot change title of News tool", which consists of 32 characters. This is below the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Title of News tool remains unchanged after several update attempts"

This revised summary is 66 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled',
'Description':'If the player opens their inventory quickly after respawning in a world with 'Immediate Respawn' enabled, the world will crash. This happened accidentally once in one of my worlds. I tried it a few more times after reloading, and it seemed to happen reliably as long as I opened my inventory at the right time.',
'VersionsAffected':'PS4 1.89',
'IssueType':'Bug',
'Project':'Minecraft Console Edition',
'Components':null}}""",
        "output": """The current summary is "Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled", which consists of 90 characters. This exceeds the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Crash on inventory access with active 'Immediate Respawn' on respawn"

This revised summary is 68 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Epic link gets removed when moving an epic from a non next gen project to a next project',
'Description':'h3. Summary \r\n\r\n When an epic from a non next-Gen project is moved to a next-Gen project, the association is removed. All issues associated with the epics will  no longer be linked to the epic \r\n\r\n h3. Steps to reproduce \r\n\r\n # Have 2 projects (Next-gen and non next-gen) \r\n # Create an Epic from a non Next-gen project \r\n # Add a child issue to the epic \r\n # Move the epic to the next-gen project \r\n\r\n h3. Expected Result \r\n Epic link remains. The issues are still linked with the epic in the next-gen project even after the moved. \r\n\r\n h3. Actual Result \r\n Epic link is lost',
'VersionsAffected':null,
'IssueType':'Bug',
'Project':'Jira Software Cloud',
'Components':'next-gen'}}""",
        "output": """The current summary is "Epic link gets removed when moving an epic from a non next gen project to a next project", which consists of 88 characters. This exceeds the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Epic link loss on moving an epic to a 'next-gen' project"

This revised summary is 56 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    }
]

In [4]:
sumLen_FewShot_temp = PromptTemplate(
    input_variables=["minChar", "maxChar", "input", "output"], 
    template="""'Summary Length':
Your Task is to check if the ticket summary has exactly between {minChar} and {maxChar} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {input}
Answer: {output}"""
)

In [5]:
sumLen_FewShot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=sumLen_FewShot_temp,
    suffix= """'Summary Length':
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}
Answer: """,
    input_variables=["min", "max", "ticket"],
)

sumLen_FewShot_prompt.save("prompts/summary/summaryLengthPrompt_FewShot_V2.3.0.json")

#### Output format prompt

In [6]:
sumLen_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<violation_predicted>": "<Set to TRUE, if a change was detected and made, otherwise FALSE>"
"<summary_old>": "<The current and unchanged content of the summary field>"
"<summary_new>": "<The recommended and revised content of the summary field>"

If the summary was not changed, return the original and unchanged summary in the "<summary_new>" field.
Don't return any source code or code snippets.

### Context ###
Data: {revised_ticket}"""

sumLen_format_prompt = PromptTemplate(template= sumLen_format_temp, input_variables= ["revised_ticket"])

sumLen_format_prompt.save("prompts/summary/summaryLengthFormatPrompt_V1.5.0.json")

# Arbitrary Structure Prompt (Best Practice)

#### Prompt

"""Best Practice: Description Completeness
The received ticket is from type "{issue_type}".
Your Task is to check if the "Description" field of the ticket is complete.
We define a description as complete if it contains the following pieces:
{content}
Role: abstract behavior of actors in the system context; describes who uses the system.
Task: specific things that must be done to achieve goals; solution or function of the problem.
Capability: the ability of actors to achieve goals based on certain conditions and events.
Goal: a condition or a circumstance desired by stakeholders or actors; describes the problem domain or the impact of solving the problem.

The general format of a {issue_type} can be expressed as follows: 
{strucutre}
As a <Role>, I want <Task>, So that <Goal>.

If the description is complete, return the original description.
Otherwise, recommend a new description for the ticket.
"""

In [ ]:
descComp_temp = """Best Practice: Arbitrary Structure
The received ticket is from the superior type "{issue_type}".
Your Task is to check if the "Description" field of the ticket contains all components of a given structure.
The components of the desired structure are described as follows:
{structure_desc}

The desired strucutre of a {issue_type} is as follows: 
{strucutre}

If the description is conatins all components, return the original description.
If the description contains nothing, return "No description provided.".
Otherwise, recommend a new description, suitable to the desired structure for the ticket.
"""

descComp_prompt = PromptTemplate(template= descComp_temp, input_variables= ["issue_type", "structure_desc", "structure"])

descComp_prompt.save("prompts/arbitraryStructure/arbitraryStructurePrompt_V1.0.0.json")

#### Output format prompt

In [ ]:
descComp_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<violation_predicted>": "<TRUE, if a violation was detectd, otherwise FALSE>"
"<description_old>": "<Old version of the content of the description field>"
"<description_new>": "<New version of the content of the description field>"

If the description was not changed, return the original description in the "<description_new>" field.
Don't return any source code or code snippets.

### Context ###
Data: {revised_ticket}"""

descComp_format_prompt = PromptTemplate(template= descComp_format_temp, input_variables= ["revised_ticket"])

descComp_format_prompt.save("prompts/arbitraryStructure/arbitraryStructureFormatPrompt_V1.0.0.json")

# Toxic Speech Prompt (Best Practice)

#### Prompt

we find that including the term “ethnic slur” in our prompt greatly improves the performance because this term specifies the victims to be a group. Using
“offensive or toxic content” as prompt doesn’t indi- cate the target of the victim, and thus leads to poor performance. 

LLMs cannot understand “hatespeech” is targeting a specific group, and thus also leads to poor performance.

it seems that explic- itly saying “intend to offend" is useful in surfacing out the LLMs’ underlying self-diagnosing abilities

(Wang & Chang 2022)

In [ ]:
toxicSpeech_temp = """Best Practice: Toxic Speech Detection
Your task is to check the comments section for toxic speech.
If you detect toxic speech, return the comment, the author, the datetime, and give a recommendation how to updated the comment."""

toxicSpeech_prompt = PromptTemplate(template= toxicSpeech_temp, input_variables= [])

toxicSpeech_prompt.save("prompts/toxicSpeech/toxicSpeechPrompt_V1.5.0.json")

#### Output format prompt

In [ ]:
toxicSpeech_format_temp = """### Instruction ###
You receive a list of comments from a revised ticket from an issue tracker.
Return the given Data for each entry in the following format in JSON:
"comment": "<content of the original comment>",
"author": "<name of the original comment>",
"datetime": "<datetime of the original comment>",
"recommendation": "<only the revised string of the new comment>"
"reason": "<reason for the recommendation>"
If the list is empty return "no toxic speech detected".

### Context ###
Data: {revised_ticket} """

toxicSpeech_format_prompt = PromptTemplate(template= toxicSpeech_format_temp, input_variables= ["revised_ticket"])

toxicSpeech_format_prompt.save("prompts/toxicSpeech/toxicSpeechFormatPrompt_V1.2.0.json")

# Internationalization Prompt (Best Practice)

#### Prompt

In [ ]:
# Sprache variable machen
internationalization_temp = """Best Practice: Internationalization
Bug reports not written in English are often closed immediately or ignored by developers.
Your task is to check if the "Summary", "Description", or "Comments" fields of the ticket are written in English.
If some fields of the ticket are not written in English, provide a translation.
Return only the revised ticket and nothing else."""

internationalization_prompt = PromptTemplate(template= internationalization_temp, input_variables= [])

internationalization_prompt.save("prompts/internationalization/internationalizationPrompt_V1.5.0.json")

#### output format prompt

In [ ]:
internationalization_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<field>": "<content of the field>"

### Context ###
Data: {revised_ticket}"""

internationalization_format_prompt = PromptTemplate(template= internationalization_format_temp, input_variables= ["revised_ticket"])

internationalization_format_prompt.save("prompts/internationalization/internationalizationFormatPrompt_V1.2.0.json")

# Bug Report Structure Prompt

maybe more of this structure:
- https://bugzilla.mozilla.org/page.cgi?id=bug-writing.html
- https://bugs.eclipse.org/bugs/page.cgi?id=bug-writing.html

### few-shot

In [ ]:
examples = [
    {
        "input": """{{'Summary':'Sourcetree crashes',
'Description':'Source tree crashes everytime i open , I have uninstalled and installed many times',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'Git',
'CreatedDate':'2021-09-13T05:48:08.000+0000',
'ResolvedDate':'2021-11-18T15:12:59.000+0000',
'Status':'Closed',
'Priority':'Low',
'Creator':'John Doe',
'Reporter':'John Doe',
'Resolution':'Cannot Reproduce',
'IssueLinks':null,
'Labels':null,
'VersionsFixed':null,
'Assignee':null,
'TimeSpent':null,
'Comments':[
{{
'Author':'Max Mustermann',
'Created':'2021-11-18 15:12:59.861000+00:00',
'Comment':'Issue was fixed in latest Sourcetree versions. Please use Sourcetree 3.4.5 > '
}}
]
}}""",
        "output": """
1. Overview
Source tree crashes everytime i open , I have uninstalled and installed many times

2. Steps to reproduce
!not provided

3. Actual result
Source tree crashes

4. Expected result
Source tree does not crash

5. Stack Traces
!not provided

6. Build date and hardware
!not provided

7. Additional information
!not provided
""" 
    },
    {
"input": """
{{
'Summary':'Opening and closing Sourcetree causes Pageant to crash\/close',
'Description':'I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready.\r\n\r\nDidn't quite notice it at first, so not sure which version it started at, but Sourcetree now causes Pageant to crash\/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason.\r\n\r\nWhenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. Sourcetree shouldn't close or cause any other applications to crash when it opens or when it closes.\r\n\r\nIt happens on all 3 computers of mine. All running latest Windows, latest Pageant, and latest Sourcetree.',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'General',
'CreatedDate':'2021-04-26T08:26:07.000+0000',
'ResolvedDate':'2021-04-27T06:38:20.000+0000',
'Status':'Closed',
'Priority':'Low',
'Creator':'Jane Doe',
'Reporter':'Jane Doe',
'Resolution':'Duplicate',
'IssueLinks':null,
'Labels':'verified',
'VersionsFixed':null,
'Assignee':null,
'TimeSpent':null,
'Comments':[
{{
'Author':'Jane Doe',
'Created':'2021-04-26 08:41:02.391000+00:00',
'Comment':'I have tried to disable the \"Automatically start SSH agent when Sourcetree opens\" option, as I thought maybe that was what was messing with things, but that didn't change anything. Pageant still disappears from my systray when Sourcetree opens or closes.'
}},
{{
'Author':'Jane Doe',
'Created':'2021-04-26 08:42:41.095000+00:00',
'Comment':'Just noticed Sourcetree even causes Pageant to crash\/close when I close the Options dialog... \ud83d\ude44'
}},
{{
'Author':'Jonathan Smith',
'Created':'2021-04-26 10:07:20.880000+00:00',
'Comment':'Known issue, duplicate. Please use ticket\u00a0SRCTREEWIN-13475\u00a0for suggestions and tracking updates.'
}}
]
}}""",
"output": """
1. Overview
I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready. Didn't quite notice it at first, but Sourcetree now causes Pageant to crash/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason. Whenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. It happens on all 3 computers of mine.             

2. Steps to reproduce
- Install putty and pageant outside of Sourcetree
- Set Pageant to startup during OS boot
- pen or close Sourcetree

3. Actual result
Sourcetree causes Pageant to crash\/close when it opens or closes.

4. Expected result
Sourcetree should not close or cause any other applications to crash when it opens or when it closes.

5. Stack Traces
!not provided

6. Build date and hardware
All running latest Windows, latest Pageant, and latest Sourcetree.
Not sure which version it started.

7. Additional information
- I have tried to disable the "Automatically start SSH agent when Sourcetree opens" option, as I thought maybe that was what was messing with things, but that didn't change anything. Pageant still disappears from my systray when Sourcetree opens or closes.
- Just noticed Sourcetree even causes Pageant to crash/close when I close the Options dialog.
""" 
    }
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"], 
    template="""### Context ###
A bug report is a document that describes an issue with a product or service and why it needs fixing.
The description field is significant for the bug report, as it describes the issue in detail.
The description should contain the following structure:
1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
2. Steps to reproduce: A list of steps to reproduce the error.
3. Actual result: The result that is currently produced.
4. Expected result: The result that is expected.
5. Stack Traces: A list of relevant stack traces for the bug report.
6. Build date and hardware: The build date and hardware, when and where the bug occurred.
7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Rearrange the description field of the bug report to match the structure described above.
Your outputs should correspond to those a Software Enginee would regarding the bug report.
If some information is missing, give suggestions and add them to the description field, if possible.
Return the new description field and nothing else.

The bug report is provided in JSON format.
Bug Report: {input} \n {output}"""
)

More space between the steps

Needs a cleaner look!

In [ ]:
bugReportStructure_fs_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= """### Context ###
A bug report is a document that describes an issue with a product or service and why it needs fixing.
The description field is significant for the bug report, as it describes the issue in detail.
The description should contain the following structure:
1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
2. Steps to reproduce: A list of steps to reproduce the error.
3. Actual result: The result that is currently produced.
4. Expected result: The result that is expected.
5. Stack Traces: A list of relevant stack traces for the bug report.
6. Build date and hardware: The build date and hardware, when and where the bug occurred.
7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Rearrange the description field of the bug report to match the structure described above.
Your outputs should correspond to those a Software Enginee would regarding the bug report.
Return the new description field and nothing else.

The bug report is provided in JSON format.
Bug Report: {bug_report}""",
    input_variables=["bug_report"]
)

bugReportStructure_fs_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_V2.3.0.json")

### 0-shot (deprecated)

In [ ]:
bugReportStructure_temp = """### Context ###
A bug report is a document that describes an issue with a product or service and why it needs fixing.
The description field is significant for the bug report, as it describes the issue in detail.
The description should contain the following structure:
1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
2. Steps to reproduce: A list of steps to reproduce the error.
3. Actual result: The result that is currently produced.
4. Expected result: The result that is expected.
5. Stack Traces: A list of relevant stack traces for the bug report.
6. Build date and hardware: The build date and hardware, when and where the bug occurred.
7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Rearrange the description field of the bug report to match the structure described above.
Your outputs should correspond to those a Software Enginee would regarding the bug report.
If some information is missing, give suggestions and add them to the description field, if possible.
Return the new description field and nothing else.

The bug report is provided in JSON format.
Bug Report: {bug_report}
"""

bugReportStructure_prompt = PromptTemplate(
    template=bugReportStructure_temp, 
    input_variables=["bug_report"])

bugReportStructure_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_V1.2.0.json")